In [1]:
from googleapiclient.discovery import build
from dotenv import dotenv_values
#import os
import re
import pandas as pd 
#import string

In [2]:
# Set Key Value

config = dotenv_values(".api_key")
key_value = config["KEY"]

print(key_value)

In [3]:
# Make build object for fetching from YouTube

youtube = build(
    'youtube',
    'v3',
    developerKey=key_value
)

youtube

In [4]:
# Test a basic request

request = youtube.channels().list(
    part = 'statistics',
    forUsername = 'BBCNews'
)

response = request.execute()

response

{'kind': 'youtube#channelListResponse',
 'etag': 'wCbyNq8eUZunlkDcogp8fpKL02Q',
 'pageInfo': {'totalResults': 1, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#channel',
   'etag': 'N5LKiLrrcKT705Ot1E5vD6eqEho',
   'id': 'UC16niRr50-MSBwiO3YDb3RA',
   'statistics': {'viewCount': '4971171341',
    'subscriberCount': '15300000',
    'hiddenSubscriberCount': False,
    'videoCount': '20195'}}]}

In [5]:
# Function to Get Video ID from Link 

def get_video_id(video_link):

    # Define a regular expression pattern to match YouTube video URLs
    pattern = (
        r'(?:https?://)?(?:www\.)?'
        '(?:youtube\.com/.*?[?&]v=|youtu\.be/|youtube\.com/embed/|youtube\.com/v/|youtube\.com/e/|youtube\.com/user/[^/]+/u/0/|www\.youtube\.com/user/[^/]+/u/0/|youtube\.com/s[^/]+/|www\.youtube\.com/s[^/]+/|youtube\.com/channel/|youtube\.com/c/|youtube\.com/user/[^/]+/|youtube\.com/user/[^/]+/live/|twitch\.tv/)'
        '([^"&?/ ]{11})'
    )

    # Use re.search to find the video ID in the URL
    match = re.search(pattern, video_link)

    # If a match is found, return the video ID; otherwise, return None
    return match.group(1) if match else None


"""
video_link = "https://www.youtube.com/watch?v=SIm2W9TtzR0"

id = get_video_id(video_link)
print(id)
"""


'\nvideo_link = "https://www.youtube.com/watch?v=SIm2W9TtzR0"\n\nid = get_video_id(video_link)\nprint(id)\n'

In [6]:

# Promt User for Video Link or hard code
video_link = str(input("Enter Video Link: "))
print(video_link)

video_id = get_video_id(video_link)
#video_link = https://www.youtube.com/watch?v=SIm2W9TtzR0



https://www.youtube.com/watch?v=SIm2W9TtzR0


In [7]:

# Take Comments and make pandas dataframe 
request = youtube.commentThreads().list(
    part="snippet",
    videoId=video_id,
    maxResults=100
)

response = request.execute()

comments = []

for item in response['items']:
    comment = item['snippet']['topLevelComment']['snippet']
    # Remove special characters from the comments
    comment_text = ''.join(e for e in comment['textDisplay'] if (e.isalnum() or e.isspace()))
    comments.append([comment_text])

df = pd.DataFrame(comments, columns=['comment'])

#df.head(100)

print(df['comment'])


0                                         thankyou adam
1                                            good video
2     How many maximum comments can we pull at a tim...
3     Only the top level comments are returned throu...
4            How do I put the comments into spreadsheet
5     Finally got around to making the advanced vide...
6                                                   wow
7                                works with live video 
8      Great video will it also scrape Instagram videos
9     Thank you so much Great video Looking forward ...
10    Great video Can you make an explain video on h...
11    I tried to make a loop to get all the videos f...
12    Hi i Found this video to be very helpfull in m...
13                   Does this work with Youtube shorts
14    No te olvides de continuar este proyecto y agr...
15    question i39m currently using the youtube api ...
16    But it doesn39t show all the comments even cha...
17     Thank you so much Adam It is really very 

In [62]:
# Save dataframe as csv, append the comments to CSV file
df.to_csv('comments.csv', index=False, mode='a')